In [1]:
from sys import modules

IN_COLAB = 'google.colab' in modules
if IN_COLAB:
    !pip install -q ir_axioms[examples] python-terrier

In [2]:
# Start/initialize PyTerrier.
from pyterrier import started, init

if not started():
    init(tqdm="auto")

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
edition = 28
track = "deep.passages"

In [4]:
from pyterrier.datasets import get_dataset
from ir_datasets import load

dataset_name = "msmarco-passage/trec-dl-2019/judged"
dataset = get_dataset(f"irds:{dataset_name}")
ir_dataset = load(dataset_name)

In [5]:
from pathlib import Path

cache_dir = Path("cache/")
index_dir = cache_dir / "indices" / dataset_name.replace("/", "-")
result_dir = Path(
    "/mnt/ceph/storage/data-in-progress/data-research/"
    "web-search/web-search-trec/trec-system-runs"
) / f"trec{edition}" / track
result_files = list(result_dir.iterdir())

In [6]:
from pyterrier.index import IterDictIndexer

if not index_dir.exists():
    indexer = IterDictIndexer(str(index_dir.absolute()))
    indexer.index(
        dataset.get_corpus_iter(),
        fields=["text"]
    )

In [7]:
from pyterrier.text import get_text
from pyterrier.io import read_results
from pyterrier import Transformer
from tqdm.auto import tqdm

results = [
    Transformer.from_df(read_results(result_file)) >> get_text(dataset, "text")
    for result_file in tqdm(result_files, desc="Load results")
]
results_names = [result_file.stem.replace("input.", "") for result_file in result_files]

Load results:   0%|          | 0/37 [00:00<?, ?it/s]

## Axiom definition
Now let's define our axiom for reranking.
In the `ir_axioms` package, we already include many great retrieval axioms that were found useful in literature.
But a single axiom alone will likely not affect our ranking much.
Therefore, it is essential to combine multiple axioms to form new axioms that can be used for reranking.
We'll now combine a few axiom with and (`&`) to only give a score if all axioms agree.
Then we add the `OriginalAxiom` as a fallback. That way, if our axioms did not conclude a preference, we use the original ranking order as preference.

In [8]:
from ir_axioms.axiom import (
    ArgUC, QTArg, QTPArg, aSL, PROX1, PROX2, PROX3, PROX4, PROX5, TFC1, TFC3, RS_TF, RS_TF_IDF, RS_BM25, RS_PL2, RS_QL,
    LNC1, TF_LNC, LB1, STMC1, STMC1_f, STMC2, STMC2_f, AND, LEN_AND, M_AND, LEN_M_AND, DIV, LEN_DIV, M_TDC, LEN_M_TDC,
    AndAxiom, ORIG, REG, ANTI_REG
)

# Define axiom for reranker and permutations.
axiom = ~(
        AndAxiom([
            ArgUC(),  # Very slow due to network access.
            QTArg(),  # Very slow due to network access.
            QTPArg(),  # Very slow due to network access.
            aSL(),
            LNC1(),
            TF_LNC(),
            LB1(),
            PROX1(),
            PROX2(),
            PROX3(),
            PROX4(),
            PROX5(),
            REG(),
            ANTI_REG(),
            AND(),
            LEN_AND(),
            M_AND(),
            LEN_M_AND(),
            DIV(),
            LEN_DIV(),
            RS_TF(),
            RS_TF_IDF(),
            RS_BM25(),
            RS_PL2(),
            RS_QL(),
            TFC1(),
            TFC3(),
            M_TDC(),
            LEN_M_TDC(),
            STMC1(),  # Rather slow due many similarity calculations.
            STMC1_f(),  # Rather slow due many similarity calculations.
            STMC2(),
            STMC2_f(),
        ]) | ORIG()
)

In [9]:
from ir_axioms.backend.pyterrier.transformers import AxiomaticReranker

reranked = [
    result % 10 >>
    AxiomaticReranker(
        axiom=axiom,
        index=index_dir,
        dataset=dataset_name,
        verbose=True
    ) ^ result
    for result in results
]
reranked_names = [f"{name} reranked" for name in results_names]

In [10]:
from pyterrier.pipelines import Experiment
from ir_measures import nDCG, MAP

experiment = Experiment(
    [*results, *reranked],
    dataset.get_topics(),
    dataset.get_qrels(),
    [nDCG @ 5, nDCG @ 10, nDCG @ 20, MAP],
    [*results_names, *reranked_names],
    verbose=True,
)
experiment.sort_values(by="nDCG@10", ascending=False, inplace=True)

pt.Experiment:   0%|          | 0/74 [00:00<?, ?system/s]

Reranking query axiomatically:   0%|          | 0/43 [00:00<?, ?query/s]

Reranking query axiomatically:   0%|          | 0/43 [00:00<?, ?query/s]

Reranking query axiomatically:   0%|          | 0/43 [00:00<?, ?query/s]

Reranking query axiomatically:   0%|          | 0/43 [00:00<?, ?query/s]

...

Reranking query axiomatically:   0%|          | 0/43 [00:00<?, ?query/s]

01:22:42.319 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading lookup file directly from disk
01:22:42.404 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading data file directly from disk


Reranking query axiomatically:   0%|          | 0/43 [00:00<?, ?query/s]

01:23:20.835 [main] ERROR org.terrier.structures.Index - Couldn't load an index structure called document
java.lang.OutOfMemoryError: Java heap space
01:23:20.958 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading lookup file directly from disk
01:23:21.003 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading data file directly from disk
01:23:21.054 [main] ERROR org.terrier.structures.Index - Couldn't load an index structure called document
java.lang.OutOfMemoryError: Java heap space
01:23:21.103 [main] ERROR org.terrier.structures.Index - Couldn't load an index structure called document
java.lang.OutOfMemoryError: Java heap space


JavaException: JVM exception occurred: Could not load an index for ref /home/heinrich/Development/webis/ir_axioms/examples/cache/indices/msmarco-passage-trec-dl-2019-judged, even though IndexLoader org.terrier.structures.IndexOnDisk$DiskIndexLoader could support that type of index. It may be your ref had a wrong location; Terrier logs may have more information. java.lang.IllegalArgumentException

In [ ]:
experiment